# Content-Based Recommender System

A recommender system is an algorithm or model that takes in information about a user and suggests an item—new to them—that is likely to be of interest. There are several approaches to building such a system, and this notebook will focus on content-based methods.



## Introduction



Content-Based Recommender Systems use the metadata of the items and users to give recommendations. In this notebook, the system will recommend movies to watch, so some examples of movie metadata could be movie genres, the name of the director, the production company, popularity, etc. At the same time, users metadata could be use, which include movies previously watched, age and the country they are from.

The main advantage of this approach is that in Cold-Start Scenarios—i.e. when there is still not enough user interaction—the system is still capable of giving sensible recommendations. This is due to the fact that metadata is already known—at least for the movies. However, this method does not compute similarity between the users, so the ones with a higher correlation cannot be given a stronger impact in the decision.

For the approaches to build a system like this, the options vary depending on the metadata to use or the computation power available.



### Jaccard and Cosine Similarities
> A simple approach and valid when the metadata can be treated as tags, which is common.





These two are popular similarity metrics that are useful when comparing metadata of various items. For both of them, the first step is to make a matrix $\underline{\underline{M}}$ of pairs (movie, tags), where the elements $m_{ij}$ are booleans indicating if the movie *i* has the tag *j*. For example, let's say that there is a DataFrame with two columns, the first the name of the film and the second one of its genres:

<center>

| **Movie** | **Genre** |
|:---------:|:---------:|
|     A     |     x     |
|     A     |     y     |
|     A     |     z     |
|     B     |     x     |
|     B     |     g     |

</center>

To transform it into the $\underline{\underline{M}}$ matrix, the movies will have to turn into rows and the genres into columns, like such:

<center>

| **Movie** \\ **Genre** | **x** | **y** | **z** | **g** |
|------------------------|-------|-------|-------|-------|
| **A**                  | 1     | 1     | 1     | 0     |
| **B**                  | 1     | 0     | 0     | 1     |

</center>

Consequently, the elements turn into indicators that show whether the corresponding movie falls under the listed genres. Finally, this allows the movies to be represented as vector—of the space of genres:

$$\underline{A} = (1,1,1,0) \;\text{ and }\; \underline{B} = (1,0,0,1)$$

Now that the movies are expressed as vector, the similarity measure is quite straightforward, as show the following equations:

+ **Jaccard Similarity**:   $J(\underline{A}, \underline{B}) = \frac{\underline{A}\cap\underline{B}}{\underline{A}\cup\underline{B}}$

+ **Cosine Similarity**:    $Sim(\underline{B},\underline{B}) = \frac{\underline{A}\cdot\underline{B}}{|\underline{A}||\underline{B}|}$

Coming back to the previous example, the results would be as follows:
+ **Jaccard Similarity**:   $J(\underline{A}, \underline{B}) = \frac{(1,1,1,0)\cap(1,0,0,1)}{(1,1,1,0)\cup(1,0,0,1)} = \frac{1+0+0+1}{1+1+1+1} = \frac{1}{2}$

+ **Cosine Similarity**:    $Sim(\underline{B},\underline{B}) = \frac{(1,1,1,0)\cdot(1,0,0,1)}{|(1,1,1,0)||(1,0,0,1)|} = \frac{1+0+0+1}{|\sqrt{1^2+1^2+1^2}||\sqrt{1^2+1^2}|} = \sqrt{\frac{2}{3}}$


### TF-IDF
>An appropriate approach when the metadata is text-based, which could enclose additional information not represented in tags. 


Let's say there is term *t* in a document *d*, which is part of a collection of documents—or corpus—called *D*. Then the *Term Frequency* or $TF(t,d)$ is defined as the number of times a term *t* appears in a document *d*. This can be used on its own as a way to vectorize documents, but it is very easy to **over-emphasize terms** that appear often yet **carry very little information**, such as the terms 'a', 'the' and 'of'. To counter this issue, every term should be weighted by its 'importance' or its ability to uniquely identify a specific document. 

To obtain a measure of the importance of every term, firstly it is necessary to define the *Document Frequency* or $DF(t,D)$, which is the number of documents in the corpus *D* that contain the term *t*. According to this, the higher the value of $DF(t,D)$, the lower the importance of *t*, because it doesn't **point to a specific document**. Consequently, a low value in $DF(t,D)$ would indicate that the term *t* carries special information about a particular document, thus the interest is in minimizing this value or alternatively in **maximizing the inverse** of $DF(t,D)$, which can be defined as:

$$IDF(t,D) = \log\left( \frac{|D| + 1}{DF(t,D) + 1} \right),$$

where: 

+ $|D|$ is the total number of documents in the corpus *D*.

+ The $+1$ in the denominator is a smoothing factor that avoids dividing by zero, which happens when the term *t* does not appear in any document *D*.

+ The $+1$ in the numerator is necessary to avoid computing $\log(0)$, which can happen if there is no document in the corpus *D*.

Note that this equation succeeds in representing the importance of a term *t*, because when a term *t* appears in all documents of *D*, then $|D| = DF(t,D)$, thus $IDF(t,D) = \log(1) = 0$, whereas the lower $DF(t,D)$ is, the greater the value of $IDF(t,D)$.

Finally, the *TF-IDF* measure is calculated following:

$$TFIDF(t,d,D) = TF(t,d)\, IDF(t,D).$$


#### Code application
In code, the process is similar, if done in a simplistic manner:

1. **Tokenization of text:** split sentences into individual words and stored in lists.

2. **Compute TF:** using *CountVectorizer* or *HashingTF*—more about them later—, calculate the term frequency for every word and store the resulting list in a new column.

3. **Compute IDF:** take the column where the TF is stored—i.e. previous output—and apply IDF to *scale the values* according to term importance.

4. **Computing Similarity:** with the vectorize version of the the documents via TF-IDF, similarity can be calculated like in the previous section.

However, it is possible to perform additional actions to **improve the quality of the documents**, leading to better results in a more efficient way:

1. **Text preprocessing:** cleaning the documents. This includes converting to lowercase, removing punctuation and special characters—so that for example, 'word' and 'word.' are treated equally—, and removing numeric values.

2. **Tokenization.**

3. **Removing stopwords:** common words, such as 'a', 'the' and 'of', will receive a low IDF score in most cases, so removing them just accelerates the process.

4. **Optional, but costly options:**

    + **Stemming:** reduces words to their root. E.g. 'running' turns into 'run'.

    + **Lemmatization:** finds the dictionary form of words. E.g. 'better' turns into 'good'.

5. **Computing TF.**

6. **Computing IDF.**

7. **Computing Similarity.**


#### About CountVectorizer and HashingTF

Both *CountVetorizer* and *HashingTF* can be used to generate the $TF(t,d)$ values, thus vectorizing the document based on its terms. However, they are fundamentally different approaches, starting with their definitions:

+ **CountVectorizer:** It is a feature extraction method that converts text documents into a sparse matrix of term counts. To do this, firstly it *builds a vocabulary* of all unique terms in the corpus *D* and then counts occurrences of each term in every document. This approach is useful when it is necessary to *keep track of words* and their frequencies.

+ **HashingTF:** It is a transformation that *maps terms* into a fixed-length feature vector *using a hashing function*. Instead of building a vocabulary, it applies a hash function to each term and assigns it to a predefined number of features, also called 'buckets'. This allows for *efficient computation* in exchange of interpretability, since different terms might hash to the same index (*hash collision*).

Moreover, let's discuss some specific differences between the two:

<table border="1">
    <tr>
        <th>Feature</th>
        <th>CountVectorizer</th>
        <th>HashingTF</th>
    </tr>
    <tr>
        <td><b>Interpretability</b></td>
        <td>Keeps the actual words in the vocabulary, making it easy to understand which words contribute to a document.</td>
        <td>The words are hashed, so we lose the ability to interpret which word corresponds to which feature.</td>
    </tr>
    <tr>
        <td><b>Memory Usage</b></td>
        <td>Requires storing the vocabulary, which can be large for big datasets.</td>
        <td>Uses a fixed-length vector, reducing memory requirements since it does not store the vocabulary.</td>
    </tr>
    <tr>
        <td><b>Computational Overhead</b></td>
        <td>Requires an extra step to scan the dataset and build the vocabulary before transforming text into vectors.</td>
        <td>Directly transforms text using a hashing function, i.e. it can transform the documents in one pass.</td>
    </tr>
    <tr>
        <td><b>Handling of New Words</b></td>
        <td>If a new word appears that was not in the training data, it will be ignored (unless vocabulary expansion is allowed).</td>
        <td>Can handle new words dynamically without needing retraining.</td>
    </tr>
    <tr>
        <td><b>Risk of Collisions</b></td>
        <td>No risk; each word has a unique index.</td>
        <td>Has a small chance of hash collisions where different words map to the same index, potentially reducing accuracy.</td>
    </tr>
    <tr>
        <td><b>Scalability</b></td>
        <td>May struggle with very large datasets due to vocabulary size constraints.</td>
        <td>More scalable since it does not require storing or updating a vocabulary.</td>
    </tr>
</table>


In the end, it is possible to summarize this information focusing on their use cases:

+ **CountVectorizer:** recommended if *interpretability* is important and there is enough memory to store the vocabulary.

+ **HashingTF:** more optimized for larger datasets, especially when *memory efficiency* is required, and when *new words* are expected frequently.

### Word2Vec and LSH
>A more advanced way to handle text. It has a NN architecture that can manage synonyms.



Word2Vec is based on a Neural Network and is capable of placing similar words closer together in the vector space built from the vocabulary. To do this, it takes into account the surrounding words for context, which favors the synonyms detection. Two of the possible approaches are:

+ **CBOW**: the Continuous Bag of Words model can predict words from context. It is optimal for large datasets, as it is fast and efficient, and it performs specially well with frequent words.

+ **Skip-gram**: this model can predict context from a target word. It is optimal for small datasets, due to its computational cost, but it is the best fitted to work with rare words.

After using one of the two architectures for Word2Vec, it is recommended to follow with LSH and euclidean distance, due to the way Word2Vec returns its output.

### SVD
>A dimensionality reduction technique to increase efficiency in similarity calculations.



Singular Value Decomposition or SVD is a technique that decomposes a matrix into three. For instance, let's say a matrix $\underline{\underline{A}}$ is storing information about movies like this:

<br />
<center>

|             | **Feature 1** | **Feature 2** | **Feature 3** |
|:-----------:|:-------------:|:-------------:|:-------------:|
| **Movie 1** |      0.3      |      0.1      |      0.0      |
| **Movie 2** |      0.5      |      0.0      |      0.8      |

Table 1: Example of a movie-feature matrix.

</center>
<br />

Where 'Features' refer to any way to identify a movie. Let's be more specific and work with movie genres, although the values will be arbitrary:

<br />
<center>

|             | **Space** | **Time** | **Travel** | **Alien** | **Love** |
|:-----------:|:---------:|:--------:|:----------:|:---------:|:--------:|
| **Movie 1** |     1     |     1    |      1     |     0     |     0    |
| **Movie 2** |     0     |     0    |      1     |     0     |     1    |

Table 2: A more specific example of a movie-feature matrix using genres as features.

</center>
<br />

Now that the matrix $\underline{\underline{A}}$ is defined, the basic idea of SVD is that instead of representing the movies with these 5 genres—which in turn is a five dimensional space—, they will be represented in fewer, yet more general 'Topics'. These 'Topics' could relate to the original genres as follows:

<br />
<center>

|                      | **Space** | **Time** | **Travel** | **Alien** | **Love** |
|:--------------------:|:---------:|:--------:|:----------:|:---------:|:--------:|
|  **Topic 1: Sci-Fi** |    0.7    |    0.6   |     0.8    |    0.5    |    0.0   |
| **Topic 2: Romance** |    0.0    |    0.1   |     0.5    |    0.3    |    0.9   |

Table 3: Representation of Features in the new space.

</center>
<br />

And consequently, allow this new representation:

<br />
<center>

|             | **Topic 1: Sci-Fi** | **Topic 2: Romance** |
|:-----------:|:-------------------:|:--------------------:|
| **Movie 1** |         0.9         |          0.1         |
| **Movie 2** |         0.2         |          0.8         |

Table 4: Representation of Movies in the new space.

</center>
<br />

In summary, with SVD it is possible to reduce the dimensionality of the problem at hand, making it less computationally intensive, and thus making the similarity calculation faster.

#### SVD with a more mathematical approach

Let's say that our matrix $\underline{\underline{A}}$ (movies as rows and features as columns) is a $n\times m$ matrix, meaning that it is representing $n$ movies with $m$ features. To describe the movies in a different way—with more general features, as with the 'Topics' before—, $\underline{\underline{A}}$ will be factorized in its *latent factors*.

The maximum number of latent factors is determined by $\underline{\underline{A}}$ and can be extracted from its *rank*:

$$k \equiv \text{rank}(\underline{\underline{A}}) < \min(n,m).$$

So the factorization of a matrix $\underline{\underline{A}}_{n\times m}$ and rank $k$ in its latent factors would be as follows:

$$\underline{\underline{A}}_{n\times m} = \underline{\underline{U}}_{n\times k}\; \underline{\underline{\Sigma}}_{k\times k}\; \underline{\underline{V}}_{k\times m}^T,$$

where:

+ $\underline{\underline{U}}_{n\times k}$ is the representation of the movies in the new space, i.e. Table 4.

+ $\underline{\underline{\Sigma}}_{k\times k}$ is a diagonal matrix that contains the *singular values* of the original matrix, which indicate the importance of each latent factor.

+ $\underline{\underline{V}}_{k\times m}^T$ represents the features in the new space, i.e. Table 3.

Finally, using the singular values from the matrix $\underline{\underline{\Sigma}}$ it is possible to determine which latent values are more important to the current data. This opens the possibility of truncation, meaning that there is no need to keep all $k$ latent factors, but only the most important ones to represent the data more efficiently, if only sacrificing accuracy.

## Coding a Content-Based Recommender System

In [15]:
# Load libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import helper_functions as hf
import pyspark.sql.functions as f
from pyspark.sql.types import FloatType, StringType

# Initialize Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CBRS").getOrCreate()
spark

In [16]:
# Movies Metadata (Load Dataset)
df = spark.read.parquet('data/cleaned/movies2/')
df.show(3)

+-----+--------------------+------+-----------------+--------------------+----------+--------------------+--------------------+------------+-------+----------------+--------+--------------------+--------------------+-----+------------+----------+--------+----------------------+----------------------+------------------+
|adult|              genres|    id|original_language|            overview|popularity|production_companies|production_countries|release_date|runtime|spoken_languages|  status|             tagline|               title|video|vote_average|vote_count|n_genres|n_production_companies|n_production_countries|n_spoken_languages|
+-----+--------------------+------+-----------------+--------------------+----------+--------------------+--------------------+------------+-------+----------------+--------+--------------------+--------------------+-----+------------+----------+--------+----------------------+----------------------+------------------+
|false|      [Drama, Music]|277216|  

### Jaccard and Cosine Similarities

Let's do a simple recommender system by movie genre:

In [17]:
# Explode genre list into different columns
genres = df.select('genres', 'title')
genres = df.select(df.title, f.explode(df.genres).alias('genre'))
genres.show(3)

+--------------------+-----+
|               title|genre|
+--------------------+-----+
|Straight Outta Co...|Drama|
|Straight Outta Co...|Music|
|        Viva Algeria|Drama|
+--------------------+-----+
only showing top 3 rows



In [18]:
# Create a columns of 1 to indicate a movies has a specific genre
genres = genres.withColumn('value', f.lit(1))
genres.show(3)

+--------------------+-----+-----+
|               title|genre|value|
+--------------------+-----+-----+
|Straight Outta Co...|Drama|    1|
|Straight Outta Co...|Music|    1|
|        Viva Algeria|Drama|    1|
+--------------------+-----+-----+
only showing top 3 rows



In [19]:
# Set movies as rows (groupby) and genres as columns (pivot)
# sum() makes sure it is one on the (title, genre) pairs indicated above
genres = genres.groupby('title').pivot('genre').sum('value')
genres.show(3)

+--------------------+------+---------+---------+------+-----+-----------+-----+------+-------+-------+-------+------+-----+-------+-------+---------------+--------+--------+----+-------+
|               title|Action|Adventure|Animation|Comedy|Crime|Documentary|Drama|Family|Fantasy|Foreign|History|Horror|Music|Mystery|Romance|Science Fiction|TV Movie|Thriller| War|Western|
+--------------------+------+---------+---------+------+-----+-----------+-----+------+-------+-------+-------+------+-----+-------+-------+---------------+--------+--------+----+-------+
|         We Are Many|  NULL|     NULL|     NULL|  NULL| NULL|          1| NULL|  NULL|   NULL|   NULL|   NULL|  NULL| NULL|   NULL|   NULL|           NULL|    NULL|    NULL|NULL|   NULL|
|All The Days Befo...|  NULL|     NULL|     NULL|     1| NULL|       NULL|    1|  NULL|   NULL|   NULL|   NULL|  NULL| NULL|   NULL|      1|           NULL|    NULL|    NULL|NULL|   NULL|
|      Snowman's Land|  NULL|     NULL|     NULL|     1|    

In [20]:
# Finally, set the NULLs as 0, which indicate that the movie hasn't that genre
genres = genres.fillna(0)
genres.show(3)

+--------------------+------+---------+---------+------+-----+-----------+-----+------+-------+-------+-------+------+-----+-------+-------+---------------+--------+--------+---+-------+
|               title|Action|Adventure|Animation|Comedy|Crime|Documentary|Drama|Family|Fantasy|Foreign|History|Horror|Music|Mystery|Romance|Science Fiction|TV Movie|Thriller|War|Western|
+--------------------+------+---------+---------+------+-----+-----------+-----+------+-------+-------+-------+------+-----+-------+-------+---------------+--------+--------+---+-------+
|         We Are Many|     0|        0|        0|     0|    0|          1|    0|     0|      0|      0|      0|     0|    0|      0|      0|              0|       0|       0|  0|      0|
|All The Days Befo...|     0|        0|        0|     1|    0|          0|    1|     0|      0|      0|      0|     0|    0|      0|      1|              0|       0|       0|  0|      0|
|      Snowman's Land|     0|        0|        0|     1|    1|   

This DataFrame stores the movie-genre relationships, which are required to calculate the similarities. In this scenario, it is a good idea to preemptively calculate similarities and cache them, so the recommendations can happen in real time. To do this, the first step is to turn the movies into vectors, which is an easy task considering the rows of the movie-genre DataFrame is a vector representation of the movies in the genre space. However, to save up memory, it is wise to use dense vectors, because most of the values are zeros:

In [21]:
from pyspark.ml.feature import VectorAssembler
# Train vector assembler on the dataframe
vector_assembler = VectorAssembler(inputCols=genres.columns[1:],
                                   outputCol='features')

# Create the column features with the Dense Vector representation
movie_features = vector_assembler.transform(genres).select('title', 'features')
movie_features.show(3, truncate=False)

+----------------------------+---------------------------+
|title                       |features                   |
+----------------------------+---------------------------+
|We Are Many                 |(20,[5],[1.0])             |
|All The Days Before Tomorrow|(20,[3,6,14],[1.0,1.0,1.0])|
|Snowman's Land              |(20,[3,4,17],[1.0,1.0,1.0])|
+----------------------------+---------------------------+
only showing top 3 rows



Now, to calculate the similarities for each pair of movies, let's join the resulting DataFrame with itself. This will result into a DataFrame whose rows will contain two movies—thus, two dense vectors. However, there are two things that must be guaranteed:

+ For every row, the two movies cannot be the same. This would lead to the calculation of the similarity between them, which will not do any good to the recommender system.

+ The pair (A,B) yields the same similarity than the pair (B,A), so there is no need to calculate them both.

In [22]:
# Cross Join: Cartesian Product of the tables
df_cross = movie_features.alias('left').crossJoin(movie_features.alias('right'))
df_cross.show(3)

+-----------+--------------+--------------------+--------------------+
|      title|      features|               title|            features|
+-----------+--------------+--------------------+--------------------+
|We Are Many|(20,[5],[1.0])|         We Are Many|      (20,[5],[1.0])|
|We Are Many|(20,[5],[1.0])|All The Days Befo...|(20,[3,6,14],[1.0...|
|We Are Many|(20,[5],[1.0])|      Snowman's Land|(20,[3,4,17],[1.0...|
+-----------+--------------+--------------------+--------------------+
only showing top 3 rows



To filter out the rows with the same movie twice and the repeated pairs, Python allows the use of '<' to compare text. For example:

+ 'The Hunger Games' < 'Braveheart' = False.

+ 'Braveheart' < 'The Hunger Games' = True.

Because 'B' comes before 'T', due to the alphabetical order.

In [23]:
df_cross = df_cross.filter(f.col('left.title') < f.col('right.title'))
df_cross.show(3)

+-----------+--------------+--------------------+--------------------+
|      title|      features|               title|            features|
+-----------+--------------+--------------------+--------------------+
|We Are Many|(20,[5],[1.0])|   What No One Knows|(20,[6,17],[1.0,1...|
|We Are Many|(20,[5],[1.0])|What Have They Do...|(20,[13,17],[1.0,...|
|We Are Many|(20,[5],[1.0])|     We are the tide|(20,[6,13,15],[1....|
+-----------+--------------+--------------------+--------------------+
only showing top 3 rows



Finally, the similarities will be stored in a new column and calculated via User Defined Functions (udf):

In [24]:
# --- Jaccard Similarity ---
def jaccard_similarity(v1, v2):
    # Formula:
    #   J = Intersection(A,B) / Union(A,B)
    A_and_B = sum(1 for (a,b) in zip(v1,v2) if a==1 and a==b) # Intersection
    A_or_B = sum(1 for (a,b) in zip(v1,v2) if a==1 or b==1) # Union
    return float(A_and_B) / A_or_B if A_or_B != 0 else 0.0 # Similarity

# --- Cosine Similarity ---
def cosine_similarity(v1,v2):
    # Formula:
    #   Sim = A·B / |A||B|

    # Numerator: scalar product
    num = sum(c1*c2 for (c1,c2) in zip(v1,v2))
    
    # Denominator: modules
    mod_a = np.sqrt(sum(c1**2 for c1 in v1))
    mod_b = np.sqrt(sum(c2**2 for c2 in v2))
    den = mod_a * mod_b

    # Similarity
    return float(num) / float(den) if den != 0.0 else 0.0

# --- User Defined Functions ---
jaccard_udf = f.udf(lambda v1,v2: jaccard_similarity(v1,v2), FloatType())
cosine_udf = f.udf(lambda v1,v2: cosine_similarity(v1,v2), FloatType())

In [25]:
# Apply udf to the pairs of movies in every row
df_similarities = df_cross.\
    withColumn('jaccard', jaccard_udf(f.col('left.features'), f.col('right.features'))).\
    withColumn('cosine', cosine_udf(f.col('left.features'), f.col('right.features'))).\
    select('left.title', 'right.title', 'jaccard', 'cosine')

df_similarities.show(5)

+-----------+--------------------+-------+------+
|      title|               title|jaccard|cosine|
+-----------+--------------------+-------+------+
|We Are Many|   What No One Knows|    0.0|   0.0|
|We Are Many|What Have They Do...|    0.0|   0.0|
|We Are Many|     We are the tide|    0.0|   0.0|
|We Are Many|Window Water Baby...|    1.0|   1.0|
|We Are Many|           Wood Job!|    0.0|   0.0|
+-----------+--------------------+-------+------+
only showing top 5 rows



### TF-IDF and LSH

From the dataset, only the *movie_id*, *title* and *overview* will be chosen. The terms in the overviews will yield the TF-IDF vectors.

In [26]:
overview = df.select('id', 'title', 'overview').orderBy('id').na.drop(subset=["overview"])
overview.show(3)

+---+-------------------+--------------------+
| id|              title|            overview|
+---+-------------------+--------------------+
|  2|              Ariel|Taisto Kasurinen ...|
|  3|Shadows in Paradise|An episode in the...|
|  5|         Four Rooms|It's Ted the Bell...|
+---+-------------------+--------------------+
only showing top 3 rows



Now, let's focus on transforming the data to make the process more efficient and vectorize the documents. As it was established before, this should include: preprocessing, tokenization, removing stopwords and computing both TF and IDF. *PySpark* can manage most of these, but for the preprocessing is required a specific behavior, thus the best approach would be to implement a custom stage for the pipeline.

To create a custom stage for a pipeline one option is to create an user defined function (udf), but the preprocessing stage is slightly complex, as it must convert the text to lowercase and remove any punctuation and special characters. For this reason, a more elegant solution is to use **custom Transformers**.

In [27]:
from pyspark.ml import Transformer
from pyspark.sql import DataFrame
from pyspark.ml.param.shared import HasInputCol, HasOutputCol
import re

class TextPreprocessing(Transformer, HasInputCol, HasOutputCol):
    '''
    A custom Transformer which applies preprocessing to the text: convert to lowercase and remove punctuation and special characters
    '''

    def __init__(self, input_column, output_column='clean_text'):
        super().__init__()
        
        # Define inputCol and inputOut
        self.inputCol = input_column
        self.outputCol = output_column
    
    def _transform(self, df: DataFrame) -> DataFrame:
        # Define text transformation
        def lowercase_and_punctuation(text):
            lower = text.lower() # lowercase
            return re.sub(pattern=r'[^\w\s]', repl='', string=lower) # remove all but words, digits and white spaces

        udf_preprocess = f.udf(lambda text: lowercase_and_punctuation(text))

        # Apply transformation
        return df.withColumn(
            self.outputCol, udf_preprocess(f.col('overview'))
        )

For the rest of them, the classes included in PySpark are enough to build a proper Pipeline:

In [28]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover

# Text preprocessing: convert to lowercase and remove punctuation and special characters
preprocessor = TextPreprocessing(input_column='overview', output_column='preprocessed')

# Tokenization
tokenizer = Tokenizer(inputCol="preprocessed", outputCol="words")

# Remove Stopwords
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")

# TF
count_vectorizer = CountVectorizer(
    inputCol="words", outputCol="rawFeatures", minDF=2, vocabSize=500
)

hashing_tf = HashingTF(
    inputCol='words', outputCol='rawFeatures', numFeatures=500
)

# IDF
idf = IDF(inputCol="rawFeatures", outputCol="features")

# Build a Pipeline with CountVectorizer and another with HashingTF 
pipeline_CV = Pipeline(stages=[preprocessor, tokenizer, remover, count_vectorizer, idf])
pipeline_HTF = Pipeline(stages=[preprocessor, tokenizer, remover, hashing_tf, idf])

Note that for both CountVectorizer and HashingTF—in the TF stage definition—some limits were imposed:

+ Firstly and more evident, there is a maximum number of features, which is actually much lower than the default size. This is a consequence of having a large number of movies, due to the elevate time it takes to evaluate the similarities of just one movie and the rest. To solve this issue, one option would be to make a previous selection of movies with a lighter method of based on cached information, to then apply the TF-IDF method to a smaller sample.

+ Secondly, with CountVectorizer exists the possibility of imposing conditions on the terms to be considered. For this matter, one of the most important one that can be done is requiring a term to appear in more than 1 (or more) documents, because if a term only appears in one document, then the similarity calculation of this element will always be zero—except for when a document is compared with itself. Furthermore, having terms that appear only in a few documents could lead to overfitting.

Now, let's continue by building the models to transform the text into vectors and applying them to the data:

In [29]:
# Get features by applying the pipeline
model_CV = pipeline_CV.fit(overview)
model_HTF = pipeline_HTF.fit(overview)

results_CV = model_CV.transform(overview).select('id', 'title', 'overview', 'features')
results_HTF = model_HTF.transform(overview).select('id', 'title', 'overview', 'features')

Compute similarity of a chosen movie with the rest instead of calculating it for every possible combination, as TF-IDF is a costly method:

In [30]:
# Create similarity function with a target vector
def create_similarity_function(movie_id: int, method: str = 'cosine', results: DataFrame = None):
    # Method validation
    if method not in ('cosine', 'jaccard'):
        print('Error: only "cosine" or "jaccard" methods available.')
        return None
    
    # Movie selector validation
    try:
        # Get title and vector representation
        movie = results.\
            filter(results.id == movie_id).\
            select('title', 'features').collect()[0]

        # Show movie name
        print(f'Chosen movie: {movie[0]}.')

        # Broadcast (cache) movie vector for quick access
        bc_mv = spark.sparkContext.broadcast(movie[1])

    except IndexError:
        print(f'Error: no movie with the index {movie_id} was found.')
        return None

    # Chose method to compute similarity
    if method == 'cosine':
        def sim_computation(other_movie):
            # Formula: sim = A·B /|A||B|
            num = float(np.dot(bc_mv.value, other_movie))
            
            mod_a = float(np.sqrt(np.dot(bc_mv.value, bc_mv.value)))
            mod_b = float(np.sqrt(np.dot(other_movie, other_movie)))
            den = mod_a * mod_b

            # If either A or B is an empty string, there is
            # nothing to compare, thus return no similarity (0)
            return num / den if den != 0 else 0.0
    
    else: # jaccard similarity
        def sim_computation(other_movie):
            # Formula: Sim = intersection(A,B) / Union(A,B)
            A_and_B = float(sum(1 for (a,b) in zip(bc_mv.value,other_movie) if a==1 and a==b)) # Intersection
            A_or_B = float(sum(1 for (a,b) in zip(bc_mv.value,other_movie) if a==1 or b==1)) # Union
            
            # If either A or B is an empty string, there is
            # nothing to compare, thus return no similarity (0)
            return A_and_B / A_or_B if A_or_B != 0 else 0.0 # Similarity

    return f.udf(lambda vec: sim_computation(vec), FloatType())

# Create similarity functions
example_movie_id = 22

udf_similiarity_CV = create_similarity_function(
    movie_id=example_movie_id, method='cosine', results=results_CV # <-- Specify movie id to compare
)

udf_similiarity_HTF = create_similarity_function(
    movie_id=example_movie_id, method='cosine', results=results_HTF # <-- Specify movie id to compare
)

# Compute Similarities
similarities_CV = results_CV.withColumn(
    'similarity', udf_similiarity_CV('features')
).\
    select('id', 'title', 'similarity').\
    orderBy('similarity', ascending=False).\
    filter(f.col('id') != example_movie_id)

similarities_HTF = results_HTF.withColumn(
    'similarity', udf_similiarity_HTF('features')
).\
    select('id', 'title', 'similarity').\
    orderBy('similarity', ascending=False).\
    filter(f.col('id') != example_movie_id)

Chosen movie: Pirates of the Caribbean: The Curse of the Black Pearl.
Chosen movie: Pirates of the Caribbean: The Curse of the Black Pearl.


In [33]:
import time

# Show similarity values
start = time.time()
print('--- CountVectorizer Method ---')
similarities_CV.show(truncate=False, n=5)
end = time.time()
print(f'Time elapsed: {np.round(end-start)} seconds.\n\n')

start = time.time()
print('--- HashingTF Method ---')
similarities_HTF.show(truncate=False, n=5)
end = time.time()
print(f'Time elapsed: {np.round(end-start)} seconds.')


CountVectorizer Method
+------+---------------------+----------+
|id    |title                |similarity|
+------+---------------------+----------+
|966   |The Magnificent Seven|0.43275064|
|149218|Come Dance with Me   |0.4299698 |
|206183|Bad Karma            |0.42188314|
|52505 |The Other Woman      |0.4023576 |
|36056 |The Ring             |0.4016651 |
+------+---------------------+----------+
only showing top 5 rows

Time elapsed: 85.0 seconds.

HashingTF Method
+------+--------------------+----------+
|id    |title               |similarity|
+------+--------------------+----------+
|353069|Mother's Day        |0.38411635|
|157129|Table No. 21        |0.3616478 |
|37602 |Oysters at Nam Kee's|0.35061252|
|16520 |The King and I      |0.33171204|
|361745|Autumn Dreams       |0.3298108 |
+------+--------------------+----------+
only showing top 5 rows

Time elapsed: 95.0 seconds.


The results were similar in time and in value spread, although for *HashingTF* the first recommendation is slightly clearer.

# Stop Spark Session

In [17]:
spark.stop()

# References
+ [SVD](https://machinelearningmastery.com/using-singular-value-decomposition-to-build-a-recommender-system/)

+ [TF-IDF and Word2Vec Documentation (PySpark)](https://spark.apache.org/docs/3.5.2/mllib-feature-extraction.html#tf-idf)

+ [TF-IDF with CountVectorize](https://www.analyticsvidhya.com/blog/2022/09/implementing-count-vectorizer-and-tf-idf-in-nlp-using-pyspark/)

+ [TF-IDF Example](https://runawayhorse001.github.io/LearningApacheSpark/manipulation.html)